## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nikolskoye,RU,59.7035,30.7861,23.40,98,100,4.47,light snow
1,1,Codrington,AU,-38.2667,141.9667,75.36,30,78,14.54,broken clouds
2,2,Broome,US,42.2506,-75.8330,43.86,49,1,4.00,clear sky
3,3,Ribeira Grande,PT,38.5167,-28.7000,55.80,58,20,8.05,few clouds
4,4,Constitucion,CL,-35.3333,-72.4167,60.49,56,0,19.80,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temperature_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                            (city_data_df["Max Temp"] >= min_temp)]
preferred_temperature_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Ribeira Grande,PT,38.5167,-28.7000,55.80,58,20,8.05,few clouds
4,4,Constitucion,CL,-35.3333,-72.4167,60.49,56,0,19.80,clear sky
12,12,Cape Town,ZA,-33.9258,18.4232,66.09,82,94,3.00,overcast clouds
13,13,Torbay,CA,47.6666,-52.7314,55.58,91,100,11.99,light rain
17,17,Esperance,AU,-33.8667,121.9000,54.21,63,52,3.09,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_temperature_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_temperatures_df = preferred_temperature_df.dropna()
new_temperatures_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Ribeira Grande,PT,38.5167,-28.7000,55.80,58,20,8.05,few clouds
4,4,Constitucion,CL,-35.3333,-72.4167,60.49,56,0,19.80,clear sky
12,12,Cape Town,ZA,-33.9258,18.4232,66.09,82,94,3.00,overcast clouds
13,13,Torbay,CA,47.6666,-52.7314,55.58,91,100,11.99,light rain
17,17,Esperance,AU,-33.8667,121.9000,54.21,63,52,3.09,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_temperatures_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Ribeira Grande,PT,55.80,few clouds,38.5167,-28.7000,
4,Constitucion,CL,60.49,clear sky,-35.3333,-72.4167,
12,Cape Town,ZA,66.09,overcast clouds,-33.9258,18.4232,
13,Torbay,CA,55.58,light rain,47.6666,-52.7314,
17,Esperance,AU,54.21,broken clouds,-33.8667,121.9000,
20,Ejido,VE,68.94,overcast clouds,8.5514,-71.2375,
21,Rosarito,MX,69.67,few clouds,32.3333,-117.0333,
26,Saint Peters,US,56.16,clear sky,38.8003,-90.6265,
32,Airai,TL,61.03,overcast clouds,-8.9266,125.4092,
33,New Norfolk,AU,58.66,few clouds,-42.7826,147.0587,


In [9]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Ribeira Grande,PT,55.80,few clouds,38.5167,-28.7000,Quinta da Meia Eira
4,Constitucion,CL,60.49,clear sky,-35.3333,-72.4167,Hotel Boutique Las Azucenas
12,Cape Town,ZA,66.09,overcast clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
13,Torbay,CA,55.58,light rain,47.6666,-52.7314,Gallows Cove
17,Esperance,AU,54.21,broken clouds,-33.8667,121.9000,Hospitality Esperance
...,...,...,...,...,...,...,...
651,Baikunthpur,IN,51.22,clear sky,23.2500,82.5500,Hotel Shree Annapurna
656,San Jeronimo,PE,67.08,scattered clouds,-11.9472,-75.2829,Tuki Llajta Pueblo Bonito Eco Lodge
659,Corrales,US,63.14,clear sky,35.2378,-106.6067,Quality Inn
662,Gat,IL,68.77,overcast clouds,31.6100,34.7642,OlusHome


In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Ribeira Grande,PT,55.80,few clouds,38.5167,-28.7000,Quinta da Meia Eira
4,Constitucion,CL,60.49,clear sky,-35.3333,-72.4167,Hotel Boutique Las Azucenas
12,Cape Town,ZA,66.09,overcast clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
13,Torbay,CA,55.58,light rain,47.6666,-52.7314,Gallows Cove
17,Esperance,AU,54.21,broken clouds,-33.8667,121.9000,Hospitality Esperance
...,...,...,...,...,...,...,...
651,Baikunthpur,IN,51.22,clear sky,23.2500,82.5500,Hotel Shree Annapurna
656,San Jeronimo,PE,67.08,scattered clouds,-11.9472,-75.2829,Tuki Llajta Pueblo Bonito Eco Lodge
659,Corrales,US,63.14,clear sky,35.2378,-106.6067,Quality Inn
662,Gat,IL,68.77,overcast clouds,31.6100,34.7642,OlusHome


In [ ]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
